In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.embeddings import SentenceTransformerEncoder
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory   
from camel.types import ModelPlatformType
from camel.configs import OllamaConfig, GeminiConfig, ChatGPTConfig
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage

In [3]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
AURA_INSTANCEID = os.getenv("AURA_INSTANCEID")
AURA_INSTANCENAME = os.getenv("AURA_INSTANCENAME")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
n4j = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [52]:
#gemini_llm = ModelFactory.create(model_platform=ModelPlatformType.GEMINI,model_type="gemini-1.5-pro", model_config_dict=GeminiConfig(temperature=0.2).as_dict(),)

In [5]:
openai_llm = ModelFactory.create(model_platform=ModelPlatformType.OPENAI,model_type="gpt-4o-mini", model_config_dict=ChatGPTConfig(temperature=0.2).as_dict())

In [ ]:
# mistral = ModelFactory.create(model_platform=ModelPlatformType.OLLAMA,model_type="mistral",model_config_dict=OllamaConfig(temperature=0.2).as_dict(),)


In [7]:
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=openai_llm)

In [ ]:
embedding_instance = SentenceTransformerEncoder(model_name='all-mpnet-base-v2')  

In [9]:
from camel.storages.vectordb_storages import QdrantStorage

vector_storage = QdrantStorage(
    vector_dim=embedding_instance.get_output_dim(),
    collection_name="my first collection",
    path="storage_customized_run",
)

In [10]:
from camel.retrievers import VectorRetriever

vector_retriever = VectorRetriever(embedding_model=embedding_instance,
                                   storage=vector_storage)

In [32]:
vector_retriever.process(
    content="./data/yk-faaliyet-raporu-3c2024-tr.md", vector_storage= vector_storage, max_characters=200
)

In [34]:
query= "2025 yılında Türk Hava Yollarında büyüme bekleniyor mu?"

In [33]:
retrieved_info = vector_retriever.query(
    query=query,
    top_k=4
)
retrieved_info

[{'similarity score': '0.9283719308109077',
  'content path': '/home/mericuren/Desktop/graph_Rag/data/anadoluyatırım.md',
  'metadata': {'filetype': 'text/markdown', 'languages': ['tur']},
  'extra_info': {},
  'text': '2025 Yılında Neler Bekliyoruz?\n\nTürkiye Ekonomisi'},
 {'similarity score': '0.9086349320250523',
  'content path': '/home/mericuren/Desktop/graph_Rag/data/tacirler.md',
  'metadata': {'filetype': 'text/markdown', 'languages': ['tur']},
  'extra_info': {},
  'text': '2025 yılı, Türkiye otomotiv sektörünün elektrikli araç üretimi ve ihracatında önemli bir atılım yapmasını bekliyoruz … 2025 yılında Türkiye, otomotiv ihracatında %8 ila %10 arasında bir büyüme'},
 {'similarity score': '0.9074329711280298',
  'content path': '/home/mericuren/Desktop/graph_Rag/data/burgan.md',
  'metadata': {'filetype': 'text/markdown', 'languages': ['tur']},
  'extra_info': {},
  'text': "\uf0d8 2025 yılında Türk Lirasında reel değerlenme beklentilerimiz ve en büyük ihracat pazarımız olan A

In [42]:
elements = uio.parse_file_or_url(
    input_path="/home/mericuren/Desktop/graph_Rag/data/yk-faaliyet-raporu-3c2024-tr.md"
)
chunk_elements = uio.chunk_elements(
    chunk_type="chunk_by_title", elements=elements
)
chunk_elements

In [43]:
graph_elements = []
for chunk in chunk_elements:
    graph_element = kg_agent.run(chunk, parse_graph_elements=True)
    n4j.add_graph_elements(graph_elements=[graph_element])
    graph_elements.append(graph_element)

In [32]:
len(graph_elements)

471

In [35]:
query_element = uio.create_element_from_text(
    text=query, element_id="1"
)

ans_element = kg_agent.run(query_element, parse_graph_elements=True)

In [36]:
kg_result = []
for node in ans_element.nodes:
    n4j_query = f"""
MATCH (n {{id: '{node.id}'}})-[r]->(m)
RETURN 'Node ' + n.id + ' (label: ' + labels(n)[0] + ') has relationship ' + type(r) + ' with Node ' + m.id + ' (label: ' + labels(m)[0] + ')' AS Description
UNION
MATCH (n)<-[r]-(m {{id: '{node.id}'}})
RETURN 'Node ' + m.id + ' (label: ' + labels(m)[0] + ') has relationship ' + type(r) + ' with Node ' + n.id + ' (label: ' + labels(n)[0] + ')' AS Description
"""
    result = n4j.query(query=n4j_query)
    kg_result.extend(result)

kg_result = [item['Description'] for item in kg_result]
print(kg_result)

['Node Türk Hava Yolları (label: Organization) has relationship OPERATESIN with Node Havacılık (label: Sector)', 'Node Türk Hava Yolları (label: Organization) has relationship OPERATESIN with Node uluslararası pazarlar (label: Market)', 'Node Türk Hava Yolları (label: Organization) has relationship IMPLEMENTS with Node dijital dönüşüm stratejileri (label: Strategy)', 'Node Türk Hava Yolları (label: Organization) has relationship INCREASES with Node kargo operasyonları (label: Activity)', 'Node Türk Hava Yolları (label: Organization) has relationship INVOLVEDIN with Node turizm (label: Industry)', 'Node Türk Hava Yolları (label: Organization) has relationship COMPAREDTO with Node BIST-100 (label: Index)', 'Node Türk Hava Yolları (label: Organization) has relationship IMPROVES with Node akaryakıt giderleri (label: Expense)', 'Node Türk Hava Yolları (label: Organization) has relationship IMPROVES with Node personel giderleri (label: Expense)', 'Node Türk Hava Yolları (label: Organization)

In [40]:
comined_results = str(retrieved_info) + "\n".join(kg_result)

In [ ]:
sys_msg = BaseMessage.make_assistant_message(
    role_name="Yatırım Stratejisti",
    content="""Yatırım Stratejisti, yatırım kararlarınızı desteklemek için piyasa beklentileri, finansal veriler ve sektörel trendler hakkında analizler sunar. 
Orijinal sorgunuz ve ilgili bağlam doğrultusunda, detaylı ve bilgilendirici cevaplar sağlar.
        Size orijinal Sorguyu ve Alınan Bağlamı vereceğim,
    Alınan Bağlama göre Orijinal Sorguyu cevaplayın."""
)

camel_agent = ChatAgent(system_message=sys_msg,
                        model=openai_llm)

user_prompt=f"""
Orijinal Sorgu: {query}
Bağlam: {comined_results}
"""

user_msg = BaseMessage.make_user_message(
    role_name="User", content=user_prompt
)

# Get response
agent_response = camel_agent.step(user_msg)

print(agent_response.msg.content)

2025 yılında Türk Hava Yolları'nın büyüme bekleniyor. Bağlamda, Türk Hava Yolları'nın turizmde yakaladığı olumlu ivmenin 2025'te devam edeceği ve yolcu sayısının yıllık olarak artacağı belirtiliyor. Bu durum, şirketin büyüme potansiyelini destekleyen bir faktör olarak öne çıkıyor.
